In [3]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import * 

from numpy import *

# Binary System of Sirius

---
## Question:
### How will the orbit be like if Sirius B experiences a sudden accleration?
---

We pick a binary star system that we are most familiar with - our neighbor Sirius A and Sirius B. Most stars have a companion, the gravitational force, serving as the centripital force, keeps them moving in orbit around the center of mass. When a star accelarates, the centripetal force it needs to keep in orbit would increase, so the gravitational force is not adequate to keep the star in its original orbit, so the star would actually leave its companion. However, if we assume there is friction or radiation -- something that wastes the kineetic energy, the velocity would go down and the star would be dragged back to its companion. In this model, we would simulate the orbit when Sirius B tries to escape and get pulled back.

In [4]:
# Units 
s = UNITS.second
N = UNITS.newton
kg = UNITS.kilogram
m = UNITS.meter
km = UNITS.kilometer


<Unit('kilometer')>

In [5]:
r_0 = 2.992e9 * km
T_0 = 50 * 365 * 24 * 60**2 * s
init = State(xA = -r_0 *2/3,
             yA = 0 *km,
             vxA = 0 *km/s,
             vyA = 4 / 3*pi*r_0/T_0,
             xB = r_0*1/3,
             yB = 0 * km,
             vxB = 0 *km/s,
             vyB = - 2 * pi * r_0 /3 / T_0)

,values
xA,-1994666666.6666667 kilometer
yA,0 kilometer
vxA,0.0 kilometer / second
vyA,7.9482878568752415 kilometer / second
xB,997333333.3333334 kilometer
yB,0 kilometer
vxB,0.0 kilometer / second
vyB,-3.9741439284376208 kilometer / second


The mass of Sirius A is 2.1 times the mass of the sun, rounded to 2.
The mass of Sirius B is 0.98 times the mass of the sun, rounded to 1.
Therefore the center of mass is at 1/3 of the distance from Sirius A to B.

In [6]:
m_sun = 1.989*10e30*kg
r_sun = 6.955*10e5 *km
systemA = System(init = init,
                 G = 6.674e-5 * N / kg**2 * km**2,
                 m = 2*m_sun,
                 r = 1.71 * r_sun,
                 t_0 = 0 *s
                 )

systemB = System (init = init,
                 r= 0.008*r_sun,
                 G = 6.674e-5 * N / kg**2 * km**2,
                 m = m_sun,
                 t_0 = 0*s)

,values
init,xA -1994666666.6666667 kilometer ...
r,55640.0 kilometer
G,6.674e-05 kilometer ** 2 * newton / kilogram ** 2
m,1.9890000000000001e+31 kilogram
t_0,0 second


## To accout for the deceleration of stars, we need to consider the drag force
F_drag = 1/2 * rho * mu^2 * Cd * A


 F_darg is the drag force, which is by definition the force component in the direction of the flow velocity;
 
 rho is the is the mass density of the fluid;
 
 mu is  the flow velocity relative to the object;
 
 A is the reference area;
 
 Cd is the drag coefficient, mostly depends on the Reynolds Number.

In [23]:
def drag(state, velocity, system):
    rho = 1.67 *10e-12 * kg/km**3
    #Based on research, the average density in outer space is 1 hydrogen atom per 1 cubic cm.
    mu = velocity
    A = 2 * pi * system.r**2
 
    Cd = 2/3*rho**2
    drag_force = rho * mu**2 *Cd * A/2 * km**8/kg**2 /s**2
    
    return drag_force
    

In [24]:
drag (init, 10e7, systemA)

<Quantity(0.013797312996679462, 'kilogram * kilometer / second ** 2')>

In [25]:
def universal_gravity(state, systemA, systemB):    
    """"
    Calculating gravitational force between Sirius A and Sirius B
    Taking the center of mass as origin
    xA, yA: posistion of Sirius A 
    xB, yB: position of Sirius B
    vxA, vyA: velocity of Sirius A
    vxB, vyB: velocity of Sirius B
    """
    xA, yA, vxA, yxA, xB, yB, vxB, vyB = state
    
    posA = Vector (xA, yA)
    posB = Vector (xB, yB)
    
    rA = posA.mag
    rB = posB.mag
    directionA = posA.hat()
    directionB = posB.hat()
    forceA = systemA.G * systemA.m *systemB.m / (rA+rB)**2 * directionA
    forceB = systemB.G * systemA.m *systemB.m / (rA+rB)**2 * directionB

    return rA, rB, forceA, forceB
    

In [26]:
universal_gravity(init, systemA, systemB)

(<Quantity(1994666666.6666667, 'kilometer')>,
 <Quantity(997333333.3333334, 'kilometer')>,
 <Quantity([-5.89878525e+39  0.00000000e+00], 'newton')>,
 <Quantity([5.89878525e+39 0.00000000e+00], 'newton')>)

In [27]:
def centri_force(state, systemA, systemB):
    """
    Calculating the current cetripetal force needed, related to current velocity and radius.
    
    """
    xA, yA, vxA, vyA, xB, yB, vxB, vyB = state
    
    velA = Vector(vxA, vyA)
    velB = Vector(vxB, vyB)
    posA = Vector (xA, yA)
    posB = Vector (xB, yB)
    
    vA = velA.mag
    vB = velB.mag
    rA = posA.mag
    rB = posB.mag
    
    cenA = systemA.m * vA**2 / rA
    cenB = systemB.m * vB**2 / rB
    
    cenA = cenA * (-posA.hat())
    cenB = cenB * (-posB.hat())
    
    return cenA, cenB

In [28]:
def comparing_force():
    """
    Updating distance and velocity to keep balance by comparing the gravitational force and the centripetal force required.
    """
    
    

In [29]:
#def event_func(state):


In [30]:
def acceleration_force(state, systemA, systemB ):
    
    xA, yA, vxA, vyA, xB, yB, vxB, vyB = state
    rA, rB,  grav_A, grav_B = universal_gravity(state, systemA, systemB)
    vA = sqrt(grav_A * rA / systemA.m)
    vB = sqrt(grav_B * rB / systemB.m)
    
   
    
    vel_A = Vector (vxA, vyA)
    vel_B = Vector (vxB, vyB)
    
    delta_vA = sqrt(rA * drag(state, vA, systemA)/systemA.m)
    delta_vB = sqrt(rB * drag(state, vB, systemB)/systemB.m)
    
    vA = vA-delta_vA
    vB = vB-delta_vB
    
    vA = vA * vel_A.hat()
    vB = vB * vel_B.hat()
    
    return vA, vB

In [31]:
def slope_func(state, t, systemA, systemB):
    xA, yA, vxA, vyA, xB, yB, vxB, vyB = state
    vA, vB = acceleration_force(state, systemA, systemB)
    
    
    return vA.x, vA.y, 

C:\Miniconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in sqrt
  """
C:\Miniconda3\lib\site-packages\pint\quantity.py:1236: RuntimeWarning: invalid value encountered in sqrt
  out = uf(*mobjs)


DimensionalityError: Cannot convert from 'kilometer ** 0.5 * newton ** 0.5 / kilogram ** 0.5' ([length] / [time]) to 'kilometer ** 1.5 * newton ** 0.5 / kilogram ** 0.5 / second' ([length] ** 2 / [time] ** 2)